In [ ]:
import math
import numpy as np
from PIL import Image

def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.convert('L')
        return np.array(img)
    
def save_image(after_process_img):
    img = Image.fromarray(after_process_img, mode="L")
    img.save("处理后图片.jpg")
    


